In [2]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 11.7 MB/s eta 0:00:00


In [97]:
from konlpy.tag import Okt
import pandas as pd
import json
import time
import tqdm
from multiprocessing import Pool
from tqdm.notebook import tqdm_notebook

In [76]:
# prompt: 구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [77]:
okt = Okt()

In [ ]:
# 엑셀 파일에서 리뷰 데이터 읽어오기
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/data/3팀_리뷰데이터.xlsx')

In [78]:
with open('/content/drive/MyDrive/Colab Notebooks/SentiWord_info.json', encoding='utf-8-sig', mode='r') as f:
      SentiWord_info = json.load(f)

knu_sentiment = pd.DataFrame(SentiWord_info)

In [79]:
# 감정사전 key-value(단어 - 긍정부정값)
hash_map = {}

In [160]:
knu_sentiment[knu_sentiment['word'] == '아깝다']

,word,word_root,polarity
8272,아깝다,아깝,-1


In [148]:
# 추가할 단어 있으면 추가하기
new_row = {'word': '불친절하다', 'word_root': '불친절하다', 'polarity' : '-2'}
knu_sentiment = pd.concat([knu_sentiment, pd.DataFrame(new_row, index=[0])], ignore_index=True)

In [151]:
# 감정사전 key-value 값 해싱
for index, row in knu_sentiment.iterrows():
  hash_map[row['word']] = row['polarity']

In [83]:
# 리뷰값 불러와서 긍정 부정 분석
def calculate_sentiment_korean(text):

    score = 0
    words = text.split()
    for word in words:
      score += int(hash_map.get(word,0))

    if score > 0: return 1
    elif  score < 0 : return -1
    else : return 0

In [85]:
import re
# 한글만 골라내기
def remove_non_korean(text):
    if not isinstance(text, str):
        return ''
    # 한글과 특수문자를 제외한 모든 문자를 공백으로 대체
    cleaned_text = re.sub(r'[^ ㄱ-ㅣ가-힣]', '', text)
    return cleaned_text

In [111]:
# 형용사 어간 추출 함수
def extract_adjective_stems(text):
    okt = Okt()
    # 형태소 분석
    pos_results = okt.pos(text, stem=True)
    # 형용사의 어간만 추출
    adjective_stems = [morph for morph, pos in pos_results if pos == 'Adjective']
    return adjective_stems

In [87]:
# 정규화
def extract_norm(words):
    okt = Okt()
    norm = []
    for word in words:
        # 정규화
        norm = okt.normalize(words)
    return norm

In [109]:
# 어간 추출
def extract_stem(words):
    okt = Okt()
    stems = []
    # words = words.split()
    for word in words:
        # 어간 추출
        stems = okt.morphs(words,stem=True)
    return stems

In [89]:
# 리스트 문자열로 변환
def list_to_string(lst):
    return ' '.join(map(str, lst))

In [90]:
# 결측치 제거
df.dropna(inplace=True)

In [91]:
# DataFrame의 '리뷰' 컬럼에 한글만 적용
df['리뷰'] = df['리뷰'].apply(remove_non_korean)

In [92]:
# 리뷰컬럼의 값이 빈문자열이면 해당행제거
df = df[df['리뷰'].str.strip() != '']

In [112]:
from tqdm import tqdm
tqdm.pandas()
df['리뷰2'] = df['리뷰'].progress_apply(extract_adjective_stems)

100%|██████████| 115482/115482 [06:47<00:00, 283.64it/s]


In [119]:
df['리뷰3'] = df['리뷰2'].apply(list_to_string)

In [127]:
from tqdm import tqdm
tqdm.pandas()
df['리뷰3'] = df['리뷰3'].progress_apply(extract_norm)

100%|██████████| 95611/95611 [22:42<00:00, 70.17it/s]


In [129]:
df['리뷰4'] = df['리뷰3'].progress_apply(extract_stem)

100%|██████████| 95611/95611 [23:20<00:00, 68.28it/s]


In [131]:
# 리스트 -> 문자열
df['리뷰4'] = df['리뷰4'].apply(list_to_string)

In [132]:
df

,리뷰,음식유형,리뷰2,리뷰3,리뷰4
2,좋아요,디저트,[좋다],좋다,좋다
3,좋아요,디저트,[좋다],좋다,좋다
5,굳,디저트,[굳다],굳다,굳다
8,맛있게 잘 먹었습니다,디저트,[맛있다],맛있다,맛있다
11,딸기빙수 맛있어요인절미토스트 진리에요,디저트,[맛있다],맛있다,맛있다
...,...,...,...,...,...
123743,어제 당일 예약하고 방문했어요 게가 엄청 실하고 맛있었어요 올때갈때 인사도 넘 잘해...,해산물,"[실하다, 맛있다, 좋다]",실하다 맛있다 좋다,실하다 맛있다 좋다
123744,킹크랩 맛도 좋고 서비스도 좋고 괜찮았습니다,해산물,"[좋다, 좋다, 괜찮다]",좋다 좋다 괜찮다,좋다 좋다 괜찮다
123745,맛도 최고 서비스도 최고 였습니다살아 꽉찬 대게 울산가면 또 방문할꺼 같아요,해산물,[같다],같다,같다
123747,추천받아 방문했는데 왜 추천하는지 알것 같습니다 싱싱한 킹크랩에 함께 어우러져 나오...,해산물,"[같다, 싱싱하다, 맛있다, 아쉽다]",같다 싱싱하다 맛있다 아쉽다,같다 싱싱하다 맛있다 아쉽다


In [152]:
# 리뷰 데이터에 대한 감정 라벨링
from tqdm import tqdm
tqdm.pandas()
df['label'] = df['리뷰4'].apply(calculate_sentiment_korean)

In [154]:
df[df['리뷰'] == '직원은 불친절하고 일하는 사람이 부족한건지 빙수 하나 나오는데 엄청 오래걸리네요 다른 지점보다 많은 아쉬움이 느껴졌어요']

,리뷰,음식유형,리뷰2,리뷰3,리뷰4,label
286,직원은 불친절하고 일하는 사람이 부족한건지 빙수 하나 나오는데 엄청 오래걸리네요 다...,디저트,"[불친절하다, 부족하다, 많다]",불친절하다 부족하다 많다,불친절하다 부족하다 많다,-1


In [161]:
from google.colab import files

In [158]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/라벨링결과_v2.csv',index=False,encoding='utf-8')
files.download('/content/drive/MyDrive/Colab Notebooks/data/라벨링결과_v2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>